## Give AutoGen Teams Jupyter Notebooks for Agile Code Development (and a record of the conversation)

by [Oliver Morris](https://linkedin.com/in/olimoz)
10 December 2023

In previous work it has become clear that teams of LLMs, as enabled by MS AutoGen, produce their best solutions when iterating towards a solution. This means, proposing small pieces of code, executing them to confirm validity and progress towards a solution, then recifying before proceeding. The success of this approach is not surprising, it is both true of people (see agile methodologies) and a cause of the popularity of Jupyter notebooks. Furthermore, it is a reflection of the ReAct (reason then act) approach, which is offered as a [LangChain agent](https://python.langchain.com/docs/modules/agents/agent_types/react).

Currently, AutoGen's coding environment does not encourage iterative development. It is not stateful, the variables and functions defined in previous steps of a conversation are not easily available to later steps. However, such a stateful environment is available via [Microsoft TaskWeaver](https://github.com/microsoft/TaskWeaver), many await integration of the TaskWeaver agent into Autogen. 

Meanwhile, there is a another and simple way for an AutoGen team to code in a stateful environment, whilst also retaining the narrative of the conversation. We can present the Executor of the team as a Jupyter notebook. Conversation messages become markdown cells, code messages become code cells. As the team progress from one cell to another, the variables and functions developed in previous cells remain available to them.

Note, this approach of using the Executor to represent an environment is demonstrated in an official AutoGen example. In that example, a member of the team is a [chess board](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_chess.ipynb) for other team members to play on.

Let's get started...

In [52]:
import os

# Working directory and environment keys
os.chdir("/home/oliver/Documents/LangChain/ProductDevelopment/AutoGen")
cwd = os.getcwd()
print(cwd)


/home/oliver/Documents/LangChain/ProductDevelopment/AutoGen


In [53]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
_ = load_dotenv(find_dotenv(usecwd=True)) 

oai_config_value = os.environ.get('OAI_CONFIG_LIST')

#del os.environ['OAI_CONFIG_LIST']

In [54]:
import autogen

# get config, which is in the OAI_CONFIG_LIST.json file
config_list= autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {"gpt-4-1106-preview"} # Only use GPT4-Turbo, exclude 3.5. {"gpt-3.5-turbo-1106"} 
    }
)

In [55]:
gpt_config = {
    "seed"            : 42,  # change the seed for different trials
    "temperature"     : 0,   # 0 uses most likely token every time, highly repeatable. 1 is more creative.
    "config_list"     : config_list,
    #"request_timeout" : 5*60,
}

## Provision of a New Working Environment for the Executor

Normally the Executor is a instance of AutoGen's UserProxy class. So, we take the following approach:
- Subclass UserProxy with a new class, NotebookExecutor
- Provide the key functions to manage a notebook:
    - insert_cell
    - update_cell
    - view_cell
    - etc
- Use UserProxy's 'register_reply' method to submit code and receive responses from the Jupyter notebook

Note the environment in which this notebook executes is provided by the 'kernel_manager'. In this instance we use the 'python3' kernel which has access to the same packages installed in the conda environment executing this notebook. This solution is intended to be as simple as possible. Additional work would be required to enable to notebook cells to execute in a different environment, eg Docker, and to use the shell to install packages in that environment. 

Furthermore, Python has been hardcoded as the language for notebooks. This can be changed but would require further work to do so.

Note, this new class has only been tested within the context of this notebook. It inherits many methods from the parent class (USerProxy) which are not used in this notebook. Those methods HAVE NOT BEEN TESTED. 

In [56]:
import os
import re
import autogen
import nbformat
from queue import Empty
from typing import List, Dict, Optional,Tuple, Any, Union
from nbformat.v4 import new_notebook, new_code_cell, new_output, new_markdown_cell
from IPython.core.interactiveshell import InteractiveShell
from nbconvert.preprocessors import ExecutePreprocessor
from autogen.code_utils import (
    UNKNOWN,
    CODE_BLOCK_PATTERN,
    execute_code,
    extract_code
    )

class NotebookExecutor(autogen.UserProxyAgent):

    def __init__(self, 
                 name,
                 kernel_manager,        # kernel for executing notebook code
                 system_message,        # not used with LLM, for notes only
                 code_execution_config, 
                 nb = None,             # notebook, 
                 file_path: str = None, 
                 max_cell_output_length: int =1000,
                 timeout: int = 600,    # 10 min timeout, long but useful for clustering exercises etc.
                 function_map = None):

        super().__init__(
            name            = name,
            system_message  = system_message,
            llm_config      = False,
            human_input_mode= "NEVER",
            function_map    = function_map,
            max_consecutive_auto_reply=10,
            )

        # establish a notebook
        if nb is not None:
            self.nb = nb
        else:
            if file_path:
                self.nb = self.load_notebook(file_path)
            else:
                self.nb = new_notebook()
        self.max_cell_output_length = max_cell_output_length
        self.shell = InteractiveShell.instance()

        # notebook execution
        self._code_execution_config = code_execution_config
        self.timeout = timeout
        self.kernel_manager = kernel_manager
        self.kernel_client = self.kernel_manager.client()
        self.kernel_client.start_channels()

        # notebook replies to team
        self.register_reply(autogen.ConversableAgent, NotebookExecutor.generate_notebook_reply)
    
    def _get_cell_idx_by_uid(self, cell_uid: str) -> Union[int, None]:
        """
        Find the index of a cell in a Jupyter notebook using a unique reference in the cell's tags.

        :param cell_uid: The unique id by which the cell is identified
        :return: The index of the cell with the matching tag or -1 if no such cell
        """
        for cell_idx, cell in enumerate(self.nb.cells):
            # Check if 'tags' is in the cell's metadata and the unique_ref is in the tags
            if cell.get('id', '') == cell_uid:
                return cell_idx

        # return None if no such cell.
        return None
    
    def _get_cell_uid_by_idx(self, cell_idx: int) -> Union[int, None]:
        """
        Find the uid of a cell in a Jupyter notebook using the cell index

        :param cell_uid: The index of the cell in the notebook's cell list.
        :return: The uid of the cell at that location
        """
        if cell_idx in range(0,len(self.nb.cells)):
            cell = self.nb.cells[cell_idx]
        else:
            return f"Cell index {cell_idx} is out of range"

        # Check if 'tags' is in the cell's metadata and the unique_ref is in the tags
       
        return cell.id

    def _remove_extracted_code(self, message: str, extracted_code: List[Tuple[str, str]], CODE_BLOCK_PATTERN=CODE_BLOCK_PATTERN) -> str:
        """Remove extracted code segments from the text, including the code block identifiers.
            Such that the message retains only the text for entry into a markdown cell.
            This method most commonly used when splitting a message into the text for entry into a markdown cell and code for entry into a code cell.

            :param message: The original text to remove code from.
            :param extracted_code: The list of extracted code segments, most commonly a list with one member.
            :param CODE_BLOCK_PATTERN: The pattern used to identify code blocks.
            :return: The text with the code segments and their identifiers removed.
        """
        # First, remove the code blocks
        for _, code_segment in extracted_code:
            # Escape special characters in the code segment for use in a regular expression
            escaped_code_segment = re.escape(code_segment)

            # Create a pattern that matches the whole code block including its backtick identifiers
            full_block_pattern = rf"```.*?{escaped_code_segment}.*?```"
            
            # Use regex to remove the entire code block, including the identifiers
            message = re.sub(full_block_pattern, '', message, flags=re.DOTALL)

        # Now, remove intents and cell ids (@@@ and ^^^)
        custom_delimiters = ['@@@', '^^^']
        for delimiter in custom_delimiters:
            # Create a pattern that matches text wrapped within the delimiter
            delimiter_pattern = rf"{re.escape(delimiter)}.*?{re.escape(delimiter)}"
            
            # Use regex to remove the text including the delimiters
            message = re.sub(delimiter_pattern, '', message, flags=re.DOTALL)

        return message

    def _is_output_image(self, output):
        """
        Checks if the given output is an image.
        Expected to be used when truncating image content to preserve token window and minimise LLM charges

        :param output: The output to check.
        :return: True if the output is an image, False otherwise.
        """
        if output['output_type'] in ['execute_result', 'display_data']:
            return any(key.startswith('image/') for key in output['data'])
        return False

    def _extract_notebook_intent(self, content: str):
        """
        The 'intent' is the action, eg 'insert_cell', and the id of the cell on which this action should be taken.
        The Notebook agent must be prompted to wrap their intent and cell id so they can be easily extracted from the message
        This is similar to the approahc AutoGen takes to extract the code from a message

        :param content: the conversation message
        :return: the intent and the cell id
        
        """
        # Search for the pattern in the content
        match_intent   = re.search(re.compile(r"\^\^\^(.*?)\^\^\^"), content)
        match_cell_uid = re.search(re.compile(r"\@\@\@(.*?)\@\@\@"), content)
        
        # Extract the intent
        if match_intent:
            intent   = match_intent.group(1).strip() 
        else:
            intent   = None    

        # Extract the cell UID. 'starts'=0, 'ending'=-1
        if match_cell_uid:
            cell_uid = match_cell_uid.group(1).strip() 
        else:
            cell_uid = None
        
        return intent, cell_uid
    
    def generate_notebook_reply(self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[autogen.Agent] = None,
        config: Optional[Any] = None,
        ):

        """
        Processes messages perform notebook operations and execute code therein, based on the extracted intent.

        This function iterates through a specified number of recent messages, extracts any code blocks and text content, 
        identifies the intent (such as 'insert_cell' or 'update_cell'), and performs the corresponding operations 
        in a Jupyter notebook. The function can handle insertion of new cells (both markdown and code) and updating 
        existing cells with new content.

        :param messages: list of message dicts (aka a conversation), where each message contains content that may be code and/or text.
        :param sender  : The sender of the messages.
        :param config  : Configuration options for code execution
        :return        : is final message?, reply content

        The function first checks the configuration for code execution. If disabled, it immediately returns without 
        processing. For each message, it extracts any embedded code blocks and separate text content. It then determines 
        the intent of the message, such as inserting or updating a notebook cell. Based on this intent, it either 
        inserts new cells at the specified location in the notebook (handling both text and code cells) or updates an 
        existing cell with new content (appending text to a prior text cell if provided, and updating a specific code cell).

        The function returns a boolean indicating whether any notebook operation was performed and a message detailing 
        the outcome of the operation, such as successful insertion or update of notebook cells.
        """

        code_execution_config = config if config is not None else self._code_execution_config

        if code_execution_config is False:
            return False, None
        
        if messages is None:
            messages = self._oai_messages[sender]

        last_n_messages = code_execution_config.pop("last_n_messages", 1)

        for i in range(min(len(messages), last_n_messages)):
            message = messages[-(i + 1)]

            # if nothing here, move to next message
            if not message["content"]:
                continue

            # Extract code blocks and text content
            code_blocks = extract_code(message["content"], pattern=CODE_BLOCK_PATTERN)
            #if len(code_blocks) == 1 and code_blocks[0][0] == UNKNOWN:
            #    continue
            
            text_content = self._remove_extracted_code(message["content"], code_blocks)

            # Extract intent and cell UID
            intent, cell_uid = self._extract_notebook_intent(message["content"])

            # Perform notebook operations based on the intent
            if intent == "insert_cell":
                # Join all code blocks into one string
                code_content = '\n'.join([code for _, code in code_blocks])
                logs, code_cell_uid = self.insert_cell(cell_uid, text_content, code_content)
                if code_cell_uid is not None:
                    code_execution_output = self.execute_cell(code_cell_uid)
                else:
                    return True, f"Cell id {cell_uid} is unknown. If you intended to insert a cell at the end of the notebook, restate the request with cell_uid=ending wrapped in '@@@'. Else declare the cell id after which you intended to insert a new cell"

            elif intent == "update_cell":
                # Update a single cell with either text, code, or both
                text_to_update = text_content if text_content.strip() else None
                code_to_update = '\n'.join([code for _, code in code_blocks]) if code_blocks else None
                logs, code_cell_uid = self.update_cell(cell_uid, text_to_update, code_to_update)
                if code_cell_uid is not None:
                    code_execution_output = self.execute_cell(code_cell_uid)
                else:
                    return True, f"Cell id {cell_uid} is unknown. If you intended to update the cell at the end of the notebook, restate the request with cell_uid=ending wrapped in '@@@'. Else declare the cell id you intended to update"

            else:
                return True, "Declare your intent wrapped in a block denoted in '^^^': insert_cell, update_cell or view_cell"

            if 'code_execution_output' in locals():
                code_execution_config["last_n_messages"] = last_n_messages
                #exitcode2str = "execution succeeded" if exitcode == 0 else "execution failed"
                return True, f"{intent} Cell ID={code_cell_uid}.\n{code_execution_output})\nNotebook operation output: {logs}"

        code_execution_config["last_n_messages"] = last_n_messages

        return False, None

    def update_cell(self, cell_uid: str, text: Optional[str] = None, code: Optional[str] = None) -> Union[str, str]:
        """
        Updates a specific cell in the notebook. Can handle updating of a text cell, a code cell, or both.

        :param cell_uid: The unique id of the cell to be updated.
        :param text: The text content for updating a markdown cell (optional).
        :param code: The code content for updating a code cell (optional).
        :return: A status message indicating success or failure and the cell id
        """

        if not text and not code:
            return "No content provided for update."

        # Find the cell index from the cell UID
        cell_idx = self._get_cell_idx_by_uid(cell_uid)
        if cell_idx is None:
            return f"Cell UID {cell_uid} is unknown.", None

        status_messages = []
        cell = self.nb.cells[cell_idx]

        # Update the text cell if text is provided
        if text:
            # Find the nearest preceding markdown cell to append the text
            for idx in range(cell_idx, -1, -1):
                if cell.cell_type == 'markdown':
                    cell.source += "\n" + text
                    status_messages.append(f"Text appended to markdown cell id= {cell.id}.")
                    break
            else:
                status_messages.append("No preceding markdown cell found for text appending.")

        # Update the code cell if code is provided
        if code:
            if cell.cell_type == 'code':
                cell.source = code
                status_messages.append(f"Code cell {cell.id} updated successfully.")
            else:
                status_messages.append(f"Cell {cell.id} is not a code cell.")

        return '\n'.join(status_messages), cell.id

    def insert_cell(self, cell_uid: str, text: Optional[str] = None, code: Optional[str] = None) -> Union[str, str]:
        """
        Inserts new cells into the notebook. Can handle insertion of a text cell, a code cell, or both.

        :param cell_uid: The unique id of the cell after which the new cell(s) will be inserted.
        :param text: The text content for a markdown cell (optional).
        :param code: The code content for a code cell (optional).
        :return: A status message indicating success or failure.
        """
        code_cell_uid = None

        if not text and not code:
            return "No content provided for insertion."

        # Determine the insertion index
        if cell_uid == 'starts':
            insert_idx = 0
        elif cell_uid == 'ending':
            insert_idx = len(self.nb.cells)
        else:
            insert_idx = self._get_cell_idx_by_uid(cell_uid)
            if insert_idx is None:
                return f"Cell UID {cell_uid} is unknown.", None
            else:
                insert_idx += 1

        status_messages = []

        # Insert text cell if text content is provided
        if text:
            text_cell = new_markdown_cell(text)
            if insert_idx >= len(self.nb.cells):
                self.nb.cells.append(text_cell)
            else:
                self.nb.cells.insert(insert_idx, text_cell)

            status_messages.append(f"Markdown cell inserted to new cell, id={text_cell.id}.")
            cell_uid=text_cell.id
            insert_idx += 1  # Increment index for subsequent insertion

        # Insert code cell if code content is provided
        if code:
            code_cell = new_code_cell(code)
            if insert_idx >= len(self.nb.cells):
                self.nb.cells.append(code_cell)
            else:
                self.nb.cells.insert(insert_idx, code_cell)

            status_messages.append(f"Code cell inserted to new cell, id={code_cell.id}.")
            cell_uid=code_cell.id

        return '\n'.join(status_messages), cell_uid

    def delete_cell(self, cell_uid: int) -> str:
        """
        Deletes a cell and its associated outputs

        Intentionally requires a cell uid, not a cell idx. 
        Cell indexes are simply locations in a list and change frequently, 
        considered insufficient to delete a cell

        :param cell_uid: The id of the cell to delete.
        :return: A status message indicating success or failure.
        """

        if cell_uid =='starts':
            cell_idx = 0
        elif cell_uid =='ending':
            cell_idx = -1
        else:
            cell_idx = self._get_cell_idx_by_uid(cell_uid)
            if cell_idx is None:
                return f"Cell {cell_uid} is unknown."
    
        if cell_idx in range(-1,len(self.nb.cells)):
            del self.nb.cells[cell_idx]
            return f"Cell {cell_uid} deleted successfully."
        else:
            return f"Cell {cell_uid} is invalid at location {cell_idx}."

    def execute_cell(self, cell_uid: str) -> Tuple[bool, Any]:
        """
        Executes a single cell in the notebook and returns its output.
        This method is called by the most common 'intents', such as 'update_cell' and 'insert_cell'

        :param cell_uid: uid of the cell to be executed.
        :return: A tuple containing a boolean indicating success or failure, and the cell output or error message.
        """

        # using the cell unique id, get cell index
        if cell_uid =='starts':
            cell_idx = 0
        elif cell_uid =='ending':
            cell_idx = -1
        else:
            cell_idx = self._get_cell_idx_by_uid(cell_uid)
            if cell_idx is None:
                return f"Cell {cell_uid} is unknown."
        
        # using the index, get the cell
        cell = self.nb.cells[cell_idx]

        print(
                f"\n>>>>>>>>>> CELL ID={cell.id}. EXECUTING CODE BLOCK (language is python)...",
                flush=True,
            )

        # execute the cell in the kernel, which persists for as long as the conversation
        if cell.cell_type == 'code':
            
            self.kernel_client.execute(cell.source, allow_stdin=False)
            cell.outputs = []

            # capture the result
            while True:
                try:
                    msg = self.kernel_client.get_iopub_msg(timeout=self.timeout)
                    msg_type = msg['msg_type']
                    content = msg['content']

                    if msg_type in ['execute_result', 'display_data']:
                        # Check if the output is an image
                        if 'image/png' in content['data']:
                            # Replace image with a note
                            note = "Image output has been replaced with this note."
                            cell.outputs.append(new_output(msg_type, data={'text/plain': note}))
                        else:
                            cell.outputs.append(new_output(msg_type, data=content['data']))
                    elif msg_type == 'stream':
                        cell.outputs.append(new_output(msg_type, name=content['name'], text=content['text']))
                    elif msg_type == 'error':
                        cell.outputs.append(new_output(msg_type, ename=content['ename'], evalue=content['evalue'], traceback=content['traceback']))

                    if msg_type == 'status' and content['execution_state'] == 'idle':
                        break
                
                # handle time outs.
                except Empty:
                    print(f"ERROR Timeout waiting for output from cell: {cell.source}")
                    break  # Adjust as per requirement
            
            # we return images for display in the conversation as a note, not the full image. The full image is kept in the Notebook only.
            # This is because the image is a lot of tokens, wastes money sending it to the LLM team.
            modified_outputs = []
            for output in cell.outputs:
                if self._is_output_image(output):
                    # Replace the image with a note for the returned value
                    modified_outputs.append("Image output has been replaced with this note.")
                else:
                    modified_outputs.append(output)  # Keep other outputs unchanged

        else:
            print(f"Cell {cell.id} is not a code cell, nothing to execute")

        return modified_outputs

    def save_notebook(self, file_path: str) -> str:
        """
        Saves the current notebook to the specified folder and filename.

        :param file_path: The file path (inc file name) where the notebook is located.
        :return: A status message indicating success or failure.
        """

        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                nbformat.write(self.nb, f)
            return "Notebook saved successfully."
        except Exception as e:
            return f"Error saving notebook: {str(e)}"
    
    def view_cell(self, cell_uid: str, outputs: bool = True) -> str:
        """
        Returns the source and output of a single cell in a text readable format (not json).
        Must provide either a cell_uid (unique id) or a cell_idx (index)

        :param cell_uid: The id of the cell whose content is to be returned.
        :param outputs: includes code cell outputs in the return value
        :return: Formatted string containing the cell's source and output.
        """

        # Determine cell index based on cell_uid
        if cell_uid == 'starts':
            cell_idx = 0
        elif cell_uid == 'ending':
            cell_idx = -1
        else:
            cell_idx = self._get_cell_idx_by_uid(cell_uid)
            if cell_idx is None:
                return f"Cell {cell_uid} is unknown."

        # Get the cell based on its index
        cell = self.nb.cells[cell_idx]

        # Start building the cell content string
        cell_content = f"# {cell_idx}: CELL UID = {cell_uid} \n# ==========================\n"

        # Adding the cell source
        if cell.cell_type == 'code':
            cell_content += f"```python\n{cell.source}\n```\n"
        elif cell.cell_type == 'markdown':
            cell_content += f"{cell.source}\n"

        # Process cell outputs, if outputs are requested
        if outputs:
            for output in cell.get('outputs', []):
                if output.output_type == 'stream':
                    output_text = output.get('text', '')

                    # Truncate long output texts to preserve token window and minimise LLM charges
                    if len(output_text) > self.max_cell_output_length:
                        output_text = output_text[:self.max_cell_output_length] + "\n[Output truncated]\n"
                    cell_content += f"```\n{output_text}\n```\n"
                    
                elif output.output_type == 'error':
                    cell_content += f"Error: {output.get('ename', '')}: {output.get('evalue', '')}\n"
                    
                elif output.output_type in ['display_data', 'execute_result']:

                    # Replace image outputs with a placeholder text to preserve token window and minimise LLM charges
                    if any(key.startswith('image/') for key in output.data):
                        cell_content += "[Image output replaced]\n"
                    else:
                        output_text = output.data.get('text/plain', '')
                        
                        # Truncate long output texts to preserve token window and minimise LLM charges
                        if len(output_text) > self.max_cell_output_length:
                            output_text = output_text[:self.max_cell_output_length] + "\n[Output truncated]\n"
                        cell_content += f"```\n{output_text}\n```\n"

        return cell_content

    def execute_cells_upto_uid(self, cell_uid_max: int) -> str:
        """
        Executes all cells in the notebook up to a specified index cell_uid
        Useful if picking up from a previous session
        Expected to be called by the user before the groupchat commences
        If cell_uid == ending then entire notebook is executed to place all params and functions into memory.

        :param cell_uid_max: uid up to which cells are to be executed.
        :return: A status message indicating success or failure.
        """

        # get index of final cell for execution
        if cell_uid_max =='starts':
            cell_idx_max = 0
        elif cell_uid_max =='ending':
            cell_idx_max = len(self.nb.cells)
        else:
            cell_idx_max = self._get_cell_idx_by_uid(cell_uid_max)
            if cell_idx_max is None:
                return f"Cell {cell_uid_max} is unknown."

        # execute all code cells
        for idx in range(0,cell_idx_max):
            cell = self.nb.cells[idx]
            if cell.cell_type == 'code':
                self.execute_cell(cell.id)

        return



## Set up the team

Assitants (LLM)
- Engineer
- Data scientist
- Planner
- Critic

User Proxies (Local PC or User)
- Executor (Code Execution)
- Admin (Human)

Arrangement
- One 'groupchat'
- Managed by a 'Manager'
- Human in the loop as 'Admin'

## Prompting the Data Scientist & Executor

There are at least two approaches for enabling the team to interact with the notebook:
- Data Scientist provided with functions to execute code directly. No need of an executor.
- Data Scientist proposes code which the Executor reads and processes using the NotebookExecutor's class methods

Here we take the later approach. Handing the NotebookExecutor's functions directly to the datascientist tends to lead to that agent having a conversation with itself, executing code without critique.
The downside is that we need to carefully prompt the Executor and the DataScientist. The Data Scientist must carefully wrap their intent (insert or update), the cell they inted to update and their code in delimiters.
The Executor is not an LLM, it is simple code, so needs those standrd delimiters to extract these parameters. Happily, GPT4 based Data Scientists tend to follow their prompt closely.


In [57]:
### Assistants ###

# Assistant agent is designed to solve a task with LLM, it is a subclass of ConversableAgent
# `human_input_mode` is default to "NEVER"
# `code_execution_config` is default to False.
# This agent doesn't execute code by default, and expects the user to execute the code.

scientist = autogen.AssistantAgent(
    name="DataScientist",
    llm_config=gpt_config,
    system_message="""Data Scientist. You follow an approved plan, iteratively.

        # YOUR ROLE

        You open and inspect data then suggest how it may need to be cleaned or arranged for analysis. Pay particular attention to each data type presented to you and its potential to assist with the business objective.
        You subsequently step back to consider appropriate algorithms, you are an advocate of agile iterative approaches, so are prepared to propose and investigate multiple algorithms in the hunt for the optimal approach.

        # WORKING WITH A CRITIC

        ALWAYS propose python code to the Critic for review prior to a final version of that code. 
        Wrap ALL such proposed code MUST be deontyed by a block wrapped by three tildas, '~~~'

        The Critic will make suggestions about your proposed code. You then enhance the code accordingly. Present the enhanced code as a final version for execution, without further involvement from the Critic.
        ONLY AFTER review by the critic shuld you present the final code for execution. 
        Final code MUST be denoted by a block wrapped in three backticks, '```'
        
        # WORKING WITH JUPYTER NOTEBOOKS

            ## ALWAYS Declare Your Intent

                When presenting final code for execution be sure to tell the Notebook what your intent is, eg to insert a new code cell or update an existing code cell.
                Your intent must be stated BEFORE the final code denoted. The intent MUST be in a block wrapped in three carets, '^^^'
                List of intents available to you, you MUST select from one of these:
                ^^^ insert_cell ^^^
                ^^^ update_cell ^^^
                ^^^ view_cell ^^^

            ## ALWAYS Declare which Cell to Act Upon

                You must also tell the Notebook which cell you intend to be acted upon. The cell uid (unique id) is denoted by a block wrapped with three at signs, '@@@'
                cell_uid is a 8 character unique id which is given by the Notebook whenever a cell is inserted into the Notebook
                If you intend the next or final cell in the notebook, then the cell_uid is always "ending". eg: 
                @@@ ending @@@
                If you intend any other cell to be updated, then you must give the unique 8 character cell_uid, eg: 
                @@@ 5gh75f97 @@@

        # EXAMPLES
    
            ## To insert a cell at the end of the notebook:

                ^^^ insert_cell ^^^
                @@@ ending @@@
                ```python
                # your code goes here
                ```

            ## IF the code you inserted at the 'ending' encounters an error, ALWAYS UPDATE THE CELL with corrections, like this:

                ^^^ update_cell ^^^ 
                @@@ ending @@@
                ```python
                # your corrected code goes here
                ```
        
        # BEST PRACTICES FOR CODING

        Your code must be designed for iterative work in a Jupyter Notebook. Simply provide the final code for execution. NEVER present your code in json format.
        If you wish to assign a variable to data from file then you must write a function in python to load the data, eg via pandas, and assign it to a variable.
        Present ONLY one code block per response. NEVER more than one code block per response.
        Do not ask others to copy and paste the result. Check the execution result returned by the Notebook.
        If an error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.

        """
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=gpt_config,
    system_message="""Critic. 

    As the Critic, your role is to evaluate and provide constructive feedback on proposals for project plans and proposed code. 
    Your critiques should focus on ensuring that the plans and code align with best practices for agile and iterative development, particularly within the Jupyter notebook format.
    You must also redirect the conversation away from any team member who repeatedly posts blank comments in the group chat.

    # You NEVER write code

    # ALWAYS Review Plans Proposed by the Team Members

        Assess the feasibility, clarity, and practicality of the proposed project plans.
        Suggest improvements or alternatives that enhance agility and flexibility.
        Ensure the plans are not over prescriptive, thus support iterative development and team collaboration.
        Cease critiques after the plan is approved by the Admin.

    # ALWAYS Review Code Proposed by Team Members

        Proposed code is wrapped in three tildas "~~~", critique such code whenever it appears in the group chat. Focus on its suitability for a Jupyter notebook.
        Critique code only once, do not repeatedly comment on the same piece of code.
        Never critique code in a formal code block, which is wrapped with this marker '```' .
        This means, check for common issues, such as trying to achieve too many steps in one notebook cell, thus negating the advantages of iterative development.
        Offer specific feedback on how to split up or improve the code chunk.
        After providing your critique, prompt the team member who propsoed the code to present revised, smaller, or improved chunk of code suitable for execution in a Jupyter notebook. 
        This revision should be done independently, without further input from you, the Critic.

    # ALWAYS Review Reponses from the Executor Which are Not Errors

        Critique the progress being made, is the code returning desirable results? 
        Interpret the result. Having done so, what do the results tell us about the effectiveness of your work?

        Remember, your goal is to foster a productive, agile environment where iterative development is emphasized. 
        Your feedback should be constructive, aiming to guide the team members towards more effective and efficient work practices.
        Do not show ANY appreciation in your responses.

    """,

)


In [58]:
### USER AGENTS ###

# User agents send messages to assistants
# UserProxyAgent is a subclass of ConversableAgent 
# `human_input_mode` is set to ALWAYS by default, unless overridden. We override it for the Executor, see below
# `llm_config` is set to False.They are humans or environments, not LLM.
# Code execution is enabled by default. LLM-based auto reply is disabled by default.
# User Agents have 'function maps' to use prepared functions
# User Agents have code execution configs to setup the coding environment they execute code in
# To modify auto reply, register a method with [`register_reply`](conversable_agent#register_reply).
# To modify the way to get human input, override `get_human_input` method.
# To modify the way to execute code blocks, single code block, or function call, override `execute_code_blocks`, `run_code`, and `execute_function` methods respectively.

# Proxy to execute code
from jupyter_client import KernelManager
from nbformat.v4    import new_notebook

# instantiate a new notebook
nb = new_notebook()

# configure kernel for notebook
# note, we use "python3" which will have access to the same packages 
# which this kernel (i.e. the calling kernel) has access to
# Shell has not been tested, so best to use an environment which already has common packages installed.
kernel_manager = KernelManager(kernel_name="python3")
kernel_manager.start_kernel()

code_execution_config={"last_n_messages": 3, 
                       "work_dir"       : "paper",        
                       "use_docker"     : False,  # set to True or image name like "python:3" to use docker
                       }

# Instantiate the Notebook Executor
notebook = NotebookExecutor(
    name="Notebook",
    system_message="""Notebook. You are a Jupyter Notebook
    You examine team messages for the Notebook cell id they refer to, their intent (insert, update, view cells) and extract code to execute accordingly.""",
    code_execution_config=code_execution_config,
    nb=nb,
    kernel_manager=kernel_manager,
    #function_map = {"act_on_notebook": act_upon_notebook}
)


In [59]:

# Human in the loop

user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the AgileProjectManager to discuss the plan and with the DataScientist to discuss approaches. Plan execution needs to be approved by this admin.",
   # user proxy is typically the only agent with termination message
   is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
   code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
        "last_n_messages": 2,
    },
)


In [60]:

## ESTABLISH GROUP CHAT ##

# A group chat class that contains the following data fields:
#    - agents: a list of participating agents.
#    - messages: a list of messages in the group chat.
#    - admin_name: the name of the admin agent if there is one. Default is "Admin". KeyBoardInterrupt will make the admin agent take over.
#    - func_call_filter: whether to enforce function call filter. Default is True. When set to True and when a message is a function call suggestion,
#      the next speaker will be chosen from an agent which contains the corresponding function name in its `function_map`. That's the 'Executor' UserProxy in this example.

groupchat = autogen.GroupChat(agents    = [user_proxy, scientist, critic, notebook], 
                              admin_name= 'Admin', 
                              messages  = [], 
                              max_round = 50)

# The chat is manage dby the chat_manager, which is a subclass of ConversableAgent like any other Agent
# Therefore, it takes the llm_config
# Thereafter, it manages who the next speaker should be

manager   = autogen.GroupChatManager(groupchat  = groupchat, 
                                     llm_config = gpt_config)

In [ ]:
KickOff = """
# Project Instructions

The source data is at: '""" +cwd+"""/Data/data.xlsx' 
This is a list of applications which use AI. The client wants to understand and characterise the market, what sectors are likely to be over served and which are underserved.
The client needs a summary diagram or table which can comfortably fit on one page of A4.

The client is keen that :
(a) no preconceptions are imposed on the data, select algorithms which avoid unevidenced assumptions
(b) algorithm hyperparameters are optimised"""

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [62]:
# It is a convenience to have the kick off message in the notebook before we start the chat, a record of the query the team are answering
notebook.insert_cell(cell_uid='ending', text=KickOff)

('Markdown cell inserted to new cell, id=4e3d8326.', '4e3d8326')

In [ ]:
# Initiate Chat

user_proxy.initiate_chat(
    recipient     = manager,
    message       = KickOff,
    clear_history = True,
    silent        = False
)

In [63]:
# Save the notebook for later inspection
# Creates unique filename so this notebook can be repeatedly executed and team solutions compared
import datetime

date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"notebook_{date_time}.ipynb"
notebook.save_notebook(file_path=os.path.join(cwd, filename))

# shutdown the notebook environment
notebook.kernel_client.stop_channels()